In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch

1. Добавить ID в как параметр (предварительно преобразовать ID в значения 1, 2, 3 (могут быть большие ID) и проверить на пропуски (можно просто заполнить одним значением)
2. Добавить бусты
3. Подумать над бОльшим размером батча и идеей аккумуляции градиента
4. Один из dnn и dnn_2 удалить, из другого сделать что то другое

In [ ]:
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'

<h3>define device

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_to(data, device):
    """
    moving data to device
    :param data: data to move
    :param device: device
    :return: moved data
    """

    if isinstance(data, (list, tuple)):
        return [move_to(x, device) for x in data]
    return data.to(device, non_blocking=True)

<h3>Define help functions

In [ ]:
from scipy.stats import pearsonr

In [ ]:
def batch_loader(X, y, batch_size = 4096):
    i = 0
    while True:
        if i + batch_size >= y.shape[0]: 
            break
            
        yield X.iloc[i:i+batch_size, :], y[i:i+batch_size]
        i += batch_size

In [ ]:
def pearson_loss(x, y):
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    loss = -torch.sum(vx*vy) / (torch.sqrt(torch.sum(vx*vx)) * torch.sqrt(torch.sum(vy*vy)))
    return loss

In [ ]:
def reduce_matching(a):
    a = a[0].tolist()
    ids = list(set(a))
    ids_dict = {}
    for i in range(len(ids)):
        ids_dict[ids[i]] = i

    for i in range(len(a)):
        a[i] = ids_dict[a[i]]
    return move_to(torch.FloatTensor(a).view(-1, 1), device)

In [ ]:
def standard(a):
    a -= a.mean()
    a /= a.std()
    return a

In [ ]:
class DNN_Model(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 0
        self.model_name = 'DNN_Model_0'
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear_3 = torch.nn.Linear(deep_size+64, 128)
        self.linear_4 = torch.nn.Linear(128, 128)
        self.linear_5 = torch.nn.Linear(128, 64)
        self.linear_6 = torch.nn.Linear(64, 1)
        
        self.id_linear_1 = torch.nn.Linear(1, 16)
        self.id_linear_2 = torch.nn.Linear(16, 32)
        self.id_linear_3 = torch.nn.Linear(32, 64)
        
        self.dout = torch.nn.Dropout(p=0.5)
        self.act = torch.nn.SiLU()
    
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        ids = X_tensor[:, 0].view(1, -1)
        X_tensor = X_tensor[:, 1:]
        
        ids = standard(reduce_matching(ids))
        
        ids = self.act(self.id_linear_1(ids))
        ids = self.act(self.id_linear_2(ids))
        ids = self.act(self.id_linear_3(ids))
        
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.act(self.linear_2(tensor_X))
        
        tensor_X = torch.cat([tensor_X, ids], axis=1)
        
        tensor_X = self.act(self.linear_3(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.act(self.linear_5(tensor_X))
        tensor_X = self.linear_6(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [ ]:
class DNN_Model_1(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 1
        self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, 1)
        )
        
        self.model_name = 'DNN_Model_1'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        X_tensor = X_tensor[:, 1:]
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [ ]:
class DNN_Model_2(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 2
        self.model_name = 'DNN_Model_2'
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, 350)
        self.linear_3 = torch.nn.Linear(350, deep_size)
        self.linear_4 = torch.nn.Linear(deep_size+64, 256)
        self.linear_5 = torch.nn.Linear(256, 256)
        self.linear_6 = torch.nn.Linear(256, 128)
        self.linear_7 = torch.nn.Linear(128, 128)
        self.linear_8 = torch.nn.Linear(128, 64)
        self.linear_9 = torch.nn.Linear(64, 1)
        
        self.id_linear_1 = torch.nn.Linear(1, 32)
        self.id_linear_2 = torch.nn.Linear(32, 64)
        self.id_linear_3 = torch.nn.Linear(64, 100)
        self.id_linear_4 = torch.nn.Linear(100, 64)
        self.id_linear_5 = torch.nn.Linear(64, 64)
        
        self.dout = torch.nn.Dropout(p=0.4)
        self.act = torch.nn.SiLU()
    
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        ids = X_tensor[:, 0].view(1, -1)
        X_tensor = X_tensor[:, 1:]
        
        ids = standard(reduce_matching(ids))
        
        ids = self.act(self.id_linear_1(ids))
        ids = self.act(self.id_linear_2(ids))
        ids = self.act(self.id_linear_3(ids))
        ids = self.dout(ids)
        ids = self.act(self.id_linear_4(ids))
        ids = self.act(self.id_linear_5(ids))
        
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        
        tensor_X = torch.cat([tensor_X, ids], axis=1)
        
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.act(self.linear_5(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X_6 = self.act(self.linear_6(tensor_X))
        tensor_X = self.act(self.linear_7(tensor_X_6))
        tensor_X = self.act(self.linear_8(tensor_X + tensor_X_6))
        tensor_X = self.linear_9(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [ ]:
class DNN_Model_3(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 3
        self.linear1 = torch.nn.Linear(300, deep_size)
        
        self.linear2_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear3_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear4_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear5_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear6_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear6_2 = torch.nn.Linear(deep_size, 1)
        
        self.act = torch.nn.SiLU()
        self.dout = torch.nn.Dropout(p=0.5)
        
        self.model_name = 'DNN_Model_3'

                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        X_tensor = X_tensor[:, 1:]
        X_tensor = self.act(self.linear1(X_tensor))
        
        X_tensor = self.act(self.linear2_1(X_tensor))
        X_tensor = self.act(self.linear2_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_2 = self.act(self.linear2_3(X_tensor))
        
        
        #res_2 = X_tensor
        
        X_tensor = self.act(self.linear3_1(res_2))
        X_tensor = self.act(self.linear3_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_3 = self.act(self.linear3_3(X_tensor))
        
        
        #res_3 = X_tensor
        
        X_tensor = self.act(self.linear4_1(res_2 + res_3))
        X_tensor = self.act(self.linear4_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_4 = self.act(self.linear4_3(X_tensor))
        
        res_4 = X_tensor

        X_tensor = self.act(self.linear5_1(res_3 + res_4))
        X_tensor = self.act(self.linear5_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        X_tensor = self.act(self.linear5_3(X_tensor))
        
        X_tensor = self.act(self.linear6_1(X_tensor + res_4))
        X_tensor = self.linear6_2(X_tensor)
        
        return X_tensor
        

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [ ]:
class DNN_Model_4(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 4
        act_1 = torch.nn.SiLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, 2*deep_size),
            
            torch.nn.Dropout(p=0.4),
            
            torch.nn.Linear(2*deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, deep_size),
            act_1(),
            
            torch.nn.Dropout(p=0.4),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, 1)
            
        )
        
        self.model_name = 'DNN_Model_4'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        X_tensor = X_tensor[:, 1:]
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [ ]:
dnn_models = [DNN_Model, DNN_Model_1, DNN_Model_2, DNN_Model_3, DNN_Model_4]

In [ ]:
"""
def corr_based_coefs(a):
    corrs = np.ones(a.shape[0])
    try:  #can be errors because of {a} shape
        preds_sum = np.sum(a, axis=0)
        corrs = np.array([pearsonr( (preds_sum-a[i])/(a.shape[0]-1), a[i])[0] for i in range(a.shape[0])])
        corrs = 1.01 - corrs
    except:
        pass
    if np.nan in corrs:
        corrs = np.ones(a.shape[0])
    corrs /= np.sum(corrs)
    print(f'Stacking coefs: {corrs}')
    return corrs
"""
def corr_based_coefs(a):
    return np.ones(a.shape[0]) / a.shape[0]

<h3>Define avaraging model

In [ ]:
class StackedDNN:
    def __init__(self, n, model, min_deep=30, max_deep=40):
        self.nets = [model(np.random.randint(min_deep, max_deep)) for i in range(n)]
        self.model_name = self.nets[0].model_name
        for net in self.nets:
            net = move_to(net, device)
    
    def fit(self, X, y, iters, batch_size=4096, subset=1, float_subset=True, silent=True, logs=True, plot=False, val=False, X_val=None, y_val=None):
        
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
            
        if logs and silent:
            print(f'Net {self.nets[0].model_name}')
            
        for i in range(len(self.nets)):
    
            scores_train = []
            scores_val = []
        
            if val:
                tensor_train_val = move_to(torch.FloatTensor(X_val.values).clone(), device)
                tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
            scores_train = []
            scores_val = []

            opt = torch.optim.Adam(self.nets[i].parameters(), lr=5e-4)

            if float_subset:
                subset_ind = np.random.uniform(subset, 1)

            subset_ind = np.random.choice(batch_size, int(batch_size*subset))
            
            if not silent:
                print(f'>>> Net {i + 1}/{len(self.nets)}')
                print(f'Deep size: {self.nets[i].deep_size}')
                print()
                
                
            for epoch in range(iters):
                offset = 0
                #learning cycle
                while offset + batch_size < y.shape[0]: 
                    self.nets[i].train()
                    self.nets[i].zero_grad()
                    X_tensor = move_to(torch.FloatTensor(X.iloc[offset:offset+batch_size, :].values), device)
                    y_tensor = move_to(torch.FloatTensor(y[offset:offset+batch_size].values), device)
                    X_tensor = X_tensor[subset_ind,:]
                    y_tensor = y_tensor[subset_ind]
                    
                    loss = pearson_loss(self.nets[i].tensor_forward(X_tensor).view(-1), y_tensor)
                    loss.backward()
                    opt.step()
                    offset += batch_size

                #prints
                if val:
                    self.nets[i].eval()
                    val_loss = pearson_loss(self.nets[i].tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                    scores_val.append(float(val_loss.detach()))
                scores_train.append(float(loss.detach()))

                if not silent:
                    print(f'epoch {epoch} finished with {loss} loss')
                    if val:
                        print(f'val loss {val_loss}')
                
            if logs and silent:
                print(f'Net {i+1}/{len(self.nets)} fitted')
                
            if not silent:
                if val:
                    print(f'Finished with \n train loss: {loss}\n val loss: {val_loss}')
                else:
                    print(f'Finished with {loss} loss')
            if plot:
                if val:
                    sns.lineplot(x=range(iters), y=scores_train, label='train')
                    sns.lineplot(x=range(iters), y=scores_val, label='val')
                else:
                    sns.lineplot(scores_train)
                plt.plot()

        
    def predict(self, X):
        y_pred = np.ndarray([len(self.nets), X.shape[0]])
        for i in range(len(self.nets)):
            self.nets[i].eval()
            y_pred[i] = np.array(self.nets[i].forward(X).detach().view(-1).cpu())
        y_pred = corr_based_coefs(y_pred).dot(y_pred)
        return y_pred
    
    def save(self):
        import os
        try:
            os.mkdir(f'./{self.model_name}')
        except:
            pass
        with open(f'./{self.model_name}/config.txt', 'w') as f:
            f.write(str(self.nets[0].index) + '\n')
            for i in range(len(self.nets)):
                torch.save(self.nets[i].state_dict(), f'./{self.model_name}/model:{i}')
                f.write(str(self.nets[i].deep_size) + '\n')

    
                       
    def load(self):
        with open(f'../input/ubiquantdevelop/{self.model_name}/config.txt', 'r') as f:
            dnn_model = dnn_models[int(f.readline())]
            
            for i in range(len(self.nets)):
                self.nets[i] = dnn_model(int(f.readline()))
                self.nets[i].load_state_dict(torch.load(f'../input/ubiquantdevelop/{self.model_name}/model:{i}', map_location=torch.device('cpu')))
                self.nets[i] = move_to(self.nets[i], device)

In [ ]:
nn_model = StackedDNN(25, DNN_Model)
nn_model.load()

nn_model_1 = StackedDNN(25, DNN_Model_1)
nn_model_1.load()

nn_model_2 = StackedDNN(25, DNN_Model_2)
nn_model_2.load()

nn_model_3 = StackedDNN(25, DNN_Model_3)
nn_model_3.load()

nn_model_4 = StackedDNN(25, DNN_Model_4)
nn_model_4.load()

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor().load_model('../input/ubiquantdevelop/cat_model')

In [ ]:
"""
[0.21285271 0.00104027 0.08117644 0.03685483 0.29002245 0.3780533 ] 0.15991742305560247 0
[0.01941347 0.17994775 0.01046224 0.21947689 0.24918182 0.32151784] 0.16056038569322678 7
[0.24130301 0.14463846 0.         0.15945655 0.04148576 0.41311623] 0.16142105624327258 224
[0.04750003 0.30549264 0.00668679 0.19995492 0.02480837 0.41555726] 0.16261531048406735 420
[0.01889336 0.39285988 0.         0.19219931 0.03816722 0.35788024] 0.16275673083332923 3049
[0.01021486 0.28383039 0.         0.04533612 0.09198494 0.56863368] 0.16372416567037357 3298
[0.0156451  0.45535975 0.00129836 0.03099276 0.01310882 0.48359521] 0.16498597605265972 4169
[0.03528531 0.16024088 0.         0.14374978 0.         0.66072404] 0.16529122198607726 15474
[0.         0.028439   0.         0.19309465 0.         0.77846635] 0.1653112226804743 37251
"""

In [ ]:
coefs = np.array([0.0156451, 0.45535975, 0.00129836, 0.03099276, 0.01310882, 0.48359521])
coefs /= np.sum(coefs)

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
features = ['investment_id'] + features
for (test_df, sample_prediction_df) in iter_test:
    res = nn_model.predict(test_df[features])
    res_1 = nn_model_1.predict(test_df[features])
    res_2 = nn_model_2.predict(test_df[features])
    res_3 = nn_model_3.predict(test_df[features])
    res_4 = nn_model_4.predict(test_df[features])
    res_boost = model.predict(test_df[features])
    
    res_array = np.array([res, res_1, res_2, res_3, res_4, res_boost])

    test_df['target']  = coefs.dot(res_array)
    
    env.predict(test_df[['row_id','target']])